In [ ]:
!pip install -q google-generativeai==0.3.1

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import pickle
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import drive
import requests
import json
from IPython.display import display, HTML, Image
import time
import re
from PIL import Image as PILImage
from io import BytesIO


import google.generativeai as genai


In [ ]:
# Google Drive 마운트
drive.mount('/content/drive', force_remount=True)

#Input from User

In [ ]:
# Enter Gemini API key
GEMINI_API_KEY = input('Enter Gemini API:')
genai.configure(api_key=GEMINI_API_KEY)

# Model-related classes and functions
class ColorDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32) if y is not None else None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        return self.X[idx]

# Function to create dynamic encoder layers
def create_encoder_layers(input_dim, hidden_dims, dropout_rate):
    layers = []
    in_features = input_dim
    for hidden_dim in hidden_dims:
        layers.append(nn.Linear(in_features, hidden_dim))
        layers.append(nn.BatchNorm1d(hidden_dim))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout_rate))
        in_features = hidden_dim
    return nn.Sequential(*layers)

def create_decoder_layers(latent_dim, hidden_dims, output_dim, dropout_rate):
    layers = []
    in_features = latent_dim
    for hidden_dim in hidden_dims:
        layers.append(nn.Linear(in_features, hidden_dim))
        layers.append(nn.BatchNorm1d(hidden_dim))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout_rate))
        in_features = hidden_dim
    # Add output layer
    layers.append(nn.Linear(in_features, output_dim))
    layers.append(nn.Sigmoid())  # Output in 0~1 range
    return nn.Sequential(*layers)

# AutoEncoder model class
class ColorAutoEncoder(nn.Module):
    def __init__(self, input_dim, encoder_hidden_dims, latent_dim, decoder_hidden_dims, output_dim, dropout_rate):
        super(ColorAutoEncoder, self).__init__()

        # Create encoder
        self.encoder_layers = create_encoder_layers(input_dim, encoder_hidden_dims, dropout_rate)
        self.latent_layer = nn.Linear(encoder_hidden_dims[-1], latent_dim)
        self.latent_activation = nn.ReLU()

        # Create decoder
        self.decoder = create_decoder_layers(latent_dim, decoder_hidden_dims, output_dim, dropout_rate)

    def encode(self, x):
        x = self.encoder_layers(x)
        latent = self.latent_activation(self.latent_layer(x))
        return latent

    def forward(self, x):
        latent = self.encode(x)
        output = self.decoder(latent)
        return output

# Function to load trained model
def load_model(model_path, scaler_path, device='cpu'):
    """Load the trained model and scalers."""
    # Load scalers
    with open(scaler_path, 'rb') as f:
        scalers = pickle.load(f)
    X_scaler = scalers['X_scaler']
    y_scaler = scalers['y_scaler']

    # Load model checkpoint
    checkpoint = torch.load(model_path, map_location=device)
    model_params = checkpoint['model_params']

    # Extract model configuration parameters
    latent_dim = model_params['lat']
    encoder_layers = model_params['enc']
    decoder_layers = model_params['dec']
    dropout_rate = model_params['drop'] / 100.0

    # Configure hidden layers
    encoder_hidden_dims = [128, 64] if encoder_layers == 2 else [128, 64, 32]
    decoder_hidden_dims = [64, 128] if decoder_layers == 2 else [32, 64, 128]

    # Initialize model
    input_dim = 12  # 4 colors * RGB
    output_dim = 3  # RGB

    model = ColorAutoEncoder(
        input_dim,
        encoder_hidden_dims,
        latent_dim,
        decoder_hidden_dims,
        output_dim,
        dropout_rate
    )

    # Load saved weights
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    return model, X_scaler, y_scaler

# Function to predict color
def predict_color(model, X_scaler, y_scaler, input_values, device='cpu'):
    """Predict color based on input values."""
    # Check input format
    if isinstance(input_values, list) and len(input_values) == 12:
        input_array = np.array([input_values])
    elif isinstance(input_values, np.ndarray) and input_values.shape[-1] == 12:
        if input_values.ndim == 1:
            input_array = input_values.reshape(1, -1)
        else:
            input_array = input_values
    else:
        raise ValueError("Input must be a list with 12 values or a numpy array of shape (N, 12).")

    # Scale input
    input_scaled = X_scaler.transform(input_array)
    input_tensor = torch.tensor(input_scaled, dtype=torch.float32).to(device)

    # Predict
    with torch.no_grad():
        output_scaled = model(input_tensor).cpu().numpy()

    # Inverse scaling
    output = y_scaler.inverse_transform(output_scaled)

    return output

# Function to visualize color prediction
def visualize_prediction(input_values, predicted_color, title=None):
    """Visualize the predicted color."""
    # Clip RGB values to 0-255 range
    predicted_rgb = np.clip(predicted_color[0], 0, 255).astype(np.uint8)

    # Normalize RGB values to 0-1 range
    predicted_rgb_norm = predicted_rgb / 255.0

    # Visualization
    fig, ax = plt.subplots(figsize=(6, 3))
    ax.add_patch(plt.Rectangle((0, 0), 1, 1, color=predicted_rgb_norm))
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_xticks([])
    ax.set_yticks([])

    if title:
        plt.title(title)
    else:
        plt.title(f"Predicted Color RGB: ({predicted_rgb[0]}, {predicted_rgb[1]}, {predicted_rgb[2]})")

    plt.tight_layout()
    plt.show()

    return predicted_rgb

# Function to convert RGB to HEX
def rgb_to_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]), int(rgb[1]), int(rgb[2]))

# Function to find matching Valspar colors using Gemini API
def find_valspar_color(rgb_values):
    """
    Use Gemini API to find closest matching Valspar colors.
    """
    # Convert RGB values to HEX code
    hex_color = rgb_to_hex(rgb_values)

    # Set up Gemini model
    model = genai.GenerativeModel('gemini-2.0-flash')

    # Create prompt
    prompt = f"""
    As a color expert specializing in the Valspar paint catalog:

    I need to identify the 3 closest matches in the Valspar paint collection to a specific color:
    - RGB: ({rgb_values[0]}, {rgb_values[1]}, {rgb_values[2]})
    - HEX: {hex_color}

    Please analyze this color and identify the closest Valspar paint colors by comparing RGB values and visual similarity.

    For the 3 closest matching Valspar colors, provide:
    1. Color Name
    2. Color Code/Number
    3. RGB Values (provide exact numbers)
    4. HEX Code
    5. Brief Description (texture, mood, common uses)
    6. Similarity Score (1-10, with 10 being identical)

    Format each match clearly with all details in a consistent structure.

    Important: Only include real, currently available Valspar colors with accurate information from their catalog. Do not invent colors or data.
    """

    # API call
    print("Requesting color matching from Gemini API...")

    try:
        response = model.generate_content(prompt)
        result = response.text
        return result
    except Exception as e:
        print(f"Error during Gemini API call: {e}")
        return f"Error: {e}"

# Function to create color sample image
def create_color_sample(rgb_values, size=(200, 100)):
    """Create a color sample image from RGB values."""
    rgb_values = np.clip(rgb_values, 0, 255).astype(np.uint8)
    color = tuple(rgb_values)
    img = PILImage.new('RGB', size, color)
    return img

# Function to display Valspar color matches
def display_valspar_matches(rgb_values, gemini_response):
    """Display Gemini API response with visualization."""
    # Show original color
    original_color = create_color_sample(rgb_values)

    # Create result container
    display(HTML("<h2>Original Color</h2>"))
    display(HTML(f"<p>RGB: ({rgb_values[0]}, {rgb_values[1]}, {rgb_values[2]}), HEX: {rgb_to_hex(rgb_values)}</p>"))
    display(original_color)

    # Show Gemini response
    display(HTML("<h2>Valspar Color Matching Results</h2>"))
    display(HTML(f"<pre>{gemini_response}</pre>"))

# Function to explain input format
def explain_input_format():
    """Explain the input format."""
    print("Input format: 12 RGB values (3 values × 4 colors = 12 values)")
    print("\nValue order:")
    print("1-3: Observed RGB values (observed_R, observed_G, observed_B)")
    print("4-6: Reference red RGB values (red_R, red_G, red_B)")
    print("7-9: Reference green RGB values (green_R, green_G, green_B)")
    print("10-12: Reference blue RGB values (blue_R, blue_G, blue_B)")
    print("\nExample: [150, 100, 80, 230, 50, 35, 120, 250, 80, 0, 0, 250]")

# Main execution code
def main():
    # Check Gemini API key
    if GEMINI_API_KEY == "YOUR_GEMINI_API_KEY":
        print("Please enter your Gemini API key:")
        api_key = input()
        genai.configure(api_key=api_key)

    # Set model paths
    model_dir = '/content/drive/MyDrive/2025_COSI_149B_Project1/Teamwork/output/model/added_data/fianl_color_model_alpha0.2_b32_beta0.5_dec2_drop10_e32_enc3_gamma0.3_lat256_losscomposite_lr0.005_20250322_2225/model_e32_b32_lr0.005_lat256.pt'
    scaler_path = '/content/drive/MyDrive/2025_COSI_149B_Project1/Teamwork/output/model/added_data/fianl_color_model_alpha0.2_b32_beta0.5_dec2_drop10_e32_enc3_gamma0.3_lat256_losscomposite_lr0.005_20250322_2225/scalers_e32_b32_lr0.005_lat256.pkl'

    # Load model
    print("Loading model...")
    try:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model, X_scaler, y_scaler = load_model(model_dir, scaler_path, device)
        print(f"Model loaded successfully. Using device: {device}")
    except Exception as e:
        print(f"Error loading model: {e}")
        print("\nPlease check the model and scaler paths.")
        return

    # Explain input format
    explain_input_format()

    # Process user input
    while True:
        try:
            user_input = input("\nEnter 12 RGB values (comma-separated, or 'q' to quit): ")

            if user_input.lower() == 'q':
                break

            # Process input
            input_values = [float(x.strip()) for x in user_input.split(',')]

            if len(input_values) != 12:
                print(f"Error: 12 values required. Currently {len(input_values)} values entered.")
                continue

            # Predict
            predicted_color = predict_color(model, X_scaler, y_scaler, input_values, device)

            # Output results
            rgb_values = np.clip(predicted_color[0], 0, 255).astype(int)
            print(f"\nPredicted Color RGB: ({rgb_values[0]}, {rgb_values[1]}, {rgb_values[2]})")

            # Visualization
            visualize_prediction(input_values, predicted_color)

            # Valspar color matching
            match_valspar = 'y'
            if match_valspar.lower() == 'y':
                valspar_matches = find_valspar_color(rgb_values)
                display_valspar_matches(rgb_values, valspar_matches)

        except ValueError as e:
            print(f"Input error: {e}")
        except Exception as e:
            print(f"Error during prediction: {e}")

# Run example
if __name__ == "__main__":
    main()

# Input from Dict

In [ ]:
# Define rgb_dict (at the top of the file)
rgb_dict = {
    'circle_2.jpg': np.array([144.63, 39.627, 33.627]),
    'triangle_4.jpg': np.array([13.178, 82.888, 48.987]),
    'paint-color_1.jpg': np.array([194.97, 183.97, 165.97]),
    'pentagon_0.jpg': np.array([11.678, 39.77, 92.407])
}

In [ ]:
# Enter Gemini API key
GEMINI_API_KEY = input('Enter Gemini API:')
genai.configure(api_key=GEMINI_API_KEY)

# Model-related classes and functions
class ColorDataset(Dataset):
    def __init__(self, X, y=None):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32) if y is not None else None

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        return self.X[idx]

# Function to create dynamic encoder layers
def create_encoder_layers(input_dim, hidden_dims, dropout_rate):
    layers = []
    in_features = input_dim
    for hidden_dim in hidden_dims:
        layers.append(nn.Linear(in_features, hidden_dim))
        layers.append(nn.BatchNorm1d(hidden_dim))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout_rate))
        in_features = hidden_dim
    return nn.Sequential(*layers)

# Function to create dynamic decoder layers
def create_decoder_layers(latent_dim, hidden_dims, output_dim, dropout_rate):
    layers = []
    in_features = latent_dim
    for hidden_dim in hidden_dims:
        layers.append(nn.Linear(in_features, hidden_dim))
        layers.append(nn.BatchNorm1d(hidden_dim))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout_rate))
        in_features = hidden_dim
    # Add output layer
    layers.append(nn.Linear(in_features, output_dim))
    layers.append(nn.Sigmoid())  # Output in 0~1 range
    return nn.Sequential(*layers)

# AutoEncoder model class
class ColorAutoEncoder(nn.Module):
    def __init__(self, input_dim, encoder_hidden_dims, latent_dim, decoder_hidden_dims, output_dim, dropout_rate):
        super(ColorAutoEncoder, self).__init__()

        # Create encoder
        self.encoder_layers = create_encoder_layers(input_dim, encoder_hidden_dims, dropout_rate)
        self.latent_layer = nn.Linear(encoder_hidden_dims[-1], latent_dim)
        self.latent_activation = nn.ReLU()

        # Create decoder
        self.decoder = create_decoder_layers(latent_dim, decoder_hidden_dims, output_dim, dropout_rate)

    def encode(self, x):
        x = self.encoder_layers(x)
        latent = self.latent_activation(self.latent_layer(x))
        return latent

    def forward(self, x):
        latent = self.encode(x)
        output = self.decoder(latent)
        return output

# Function to load trained model
def load_model(model_path, scaler_path, device='cpu'):
    """Load the trained model and scalers."""
    # Load scalers
    with open(scaler_path, 'rb') as f:
        scalers = pickle.load(f)
    X_scaler = scalers['X_scaler']
    y_scaler = scalers['y_scaler']

    # Load model checkpoint
    checkpoint = torch.load(model_path, map_location=device)
    model_params = checkpoint['model_params']

    # Extract model configuration parameters
    latent_dim = model_params['lat']
    encoder_layers = model_params['enc']
    decoder_layers = model_params['dec']
    dropout_rate = model_params['drop'] / 100.0

    # Configure hidden layers
    encoder_hidden_dims = [128, 64] if encoder_layers == 2 else [128, 64, 32]
    decoder_hidden_dims = [64, 128] if decoder_layers == 2 else [32, 64, 128]

    # Initialize model
    input_dim = 12  # 4 colors * RGB
    output_dim = 3  # RGB

    model = ColorAutoEncoder(
        input_dim,
        encoder_hidden_dims,
        latent_dim,
        decoder_hidden_dims,
        output_dim,
        dropout_rate
    )

    # Load saved weights
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    return model, X_scaler, y_scaler

# Function to predict color
def predict_color(model, X_scaler, y_scaler, input_values, device='cpu'):
    """Predict color based on input values."""
    # Check input format
    if isinstance(input_values, list) and len(input_values) == 12:
        input_array = np.array([input_values])
    elif isinstance(input_values, np.ndarray) and input_values.shape[-1] == 12:
        if input_values.ndim == 1:
            input_array = input_values.reshape(1, -1)
        else:
            input_array = input_values
    else:
        raise ValueError("Input must be a list with 12 values or a numpy array of shape (N, 12).")

    # Scale input
    input_scaled = X_scaler.transform(input_array)
    input_tensor = torch.tensor(input_scaled, dtype=torch.float32).to(device)

    # Predict
    with torch.no_grad():
        output_scaled = model(input_tensor).cpu().numpy()

    # Inverse scaling
    output = y_scaler.inverse_transform(output_scaled)

    return output

# Function to visualize color prediction
def visualize_prediction(input_values, predicted_color, title=None):
    """Visualize the predicted color."""
    # Clip RGB values to 0-255 range
    predicted_rgb = np.clip(predicted_color[0], 0, 255).astype(np.uint8)

    # Normalize RGB values to 0-1 range
    predicted_rgb_norm = predicted_rgb / 255.0

    # Visualization
    fig, ax = plt.subplots(figsize=(6, 3))
    ax.add_patch(plt.Rectangle((0, 0), 1, 1, color=predicted_rgb_norm))
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_xticks([])
    ax.set_yticks([])

    if title:
        plt.title(title)
    else:
        plt.title(f"Predicted Color RGB: ({predicted_rgb[0]}, {predicted_rgb[1]}, {predicted_rgb[2]})")

    plt.tight_layout()
    plt.show()

    return predicted_rgb

# Function to convert RGB to HEX
def rgb_to_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]), int(rgb[1]), int(rgb[2]))

# Function to find matching Valspar colors using Gemini API
def find_valspar_color(rgb_values):
    """
    Use Gemini API to find closest matching Valspar colors.
    """
    # Convert RGB values to HEX code
    hex_color = rgb_to_hex(rgb_values)

    # Set up Gemini model
    model = genai.GenerativeModel('gemini-2.0-flash')

    # Create prompt
    prompt = f"""
    As a color expert specializing in the Valspar paint catalog:

    I need to identify the 3 closest matches in the Valspar paint collection to a specific color:
    - RGB: ({rgb_values[0]}, {rgb_values[1]}, {rgb_values[2]})
    - HEX: {hex_color}

    Please analyze this color and identify the closest Valspar paint colors by comparing RGB values and visual similarity.

    For the 3 closest matching Valspar colors, provide:
    1. Color Name
    2. Color Code/Number
    3. RGB Values (provide exact numbers)
    4. HEX Code
    5. Brief Description (texture, mood, common uses)
    6. Similarity Score (1-10, with 10 being identical)

    Format each match clearly with all details in a consistent structure.

    Important: Only include real, currently available Valspar colors with accurate information from their catalog. Do not invent colors or data.
    """

    # API call
    print("Requesting color matching from Gemini API...")

    try:
        response = model.generate_content(prompt)
        result = response.text
        return result
    except Exception as e:
        print(f"Error during Gemini API call: {e}")
        return f"Error: {e}"

# Function to create color sample image
def create_color_sample(rgb_values, size=(200, 100)):
    """Create a color sample image from RGB values."""
    rgb_values = np.clip(rgb_values, 0, 255).astype(np.uint8)
    color = tuple(rgb_values)
    img = PILImage.new('RGB', size, color)
    return img

# Function to display Valspar color matches
def display_valspar_matches(rgb_values, gemini_response):
    """Display Gemini API response with visualization."""
    # Show original color
    original_color = create_color_sample(rgb_values)

    # Create result container
    display(HTML("<h2>Predicted Original Color</h2>"))
    display(HTML(f"<p>RGB: ({rgb_values[0]}, {rgb_values[1]}, {rgb_values[2]}), HEX: {rgb_to_hex(rgb_values)}</p>"))
    display(original_color)

    # Show Gemini response
    display(HTML("<h2>Valspar Color Matching Result</h2>"))
    display(HTML(f"<pre>{gemini_response}</pre>"))

# Function to explain input format
def explain_input_format():
    """Explain the input format."""
    print("Input format: 12 RGB values (3 values × 4 colors = 12 values)")
    print("\nValue order:")
    print("1-3: Observed RGB values (observed_R, observed_G, observed_B)")
    print("4-6: Reference red RGB values (red_R, red_G, red_B)")
    print("7-9: Reference green RGB values (green_R, green_G, green_B)")
    print("10-12: Reference blue RGB values (blue_R, blue_G, blue_B)")

# Function to process rgb_dict - only process paint-color_1.jpg as the observed color, use others as reference
def process_rgb_dict(rgb_dict, model, X_scaler, y_scaler, device='cpu'):
    """Process only paint-color_1.jpg from rgb_dict and use others as reference colors."""
    results = {}

    # Extract reference colors
    red_reference = rgb_dict.get('circle_2.jpg', np.array([255, 0, 0]))  # Red reference
    green_reference = rgb_dict.get('triangle_4.jpg', np.array([0, 255, 0]))  # Green reference
    blue_reference = rgb_dict.get('pentagon_0.jpg', np.array([0, 0, 255]))  # Blue reference

    # Process only paint-color_1.jpg as observed color
    if 'paint-color_1.jpg' in rgb_dict:
        observed_color = rgb_dict['paint-color_1.jpg']

        # Create input vector with 12 values (observed color + 3 reference colors)
        input_values = np.concatenate([
            observed_color,
            red_reference,
            green_reference,
            blue_reference
        ])

        # Predict color
        predicted_color = predict_color(model, X_scaler, y_scaler, input_values, device)

        # Save results
        rgb_values = np.clip(predicted_color[0], 0, 255).astype(int)
        results['paint-color_1.jpg'] = {
            'input': observed_color,
            'predicted': rgb_values,
            'hex': rgb_to_hex(rgb_values)
        }

        # Visualization
        print(f"paint-color_1.jpg")
        print(f"Observed RGB: ({observed_color[0]}, {observed_color[1]}, {observed_color[2]})")
        print(f"Predicted RGB: ({rgb_values[0]}, {rgb_values[1]}, {rgb_values[2]})")
        visualize_prediction(input_values, predicted_color, title="paint-color_1.jpg Color Prediction")

        # Valspar color matching
        try:
            valspar_matches = find_valspar_color(rgb_values)
            display_valspar_matches(rgb_values, valspar_matches)
        except Exception as e:
            print(f"Error during Valspar matching: {e}")
    else:
        print("paint-color_1.jpg not found in rgb_dict.")

    return results

# Main execution code
def main():
    # Check Gemini API key
    if GEMINI_API_KEY == "YOUR_GEMINI_API_KEY":
        print("Please enter your Gemini API key:")
        api_key = input()
        genai.configure(api_key=api_key)

    # Set model paths
    model_dir = '/content/drive/MyDrive/2025_COSI_149B_Project1/Teamwork/output/model/added_data/fianl_color_model_alpha0.2_b32_beta0.5_dec2_drop10_e32_enc3_gamma0.3_lat256_losscomposite_lr0.005_20250322_2225/model_e32_b32_lr0.005_lat256.pt'
    scaler_path = '/content/drive/MyDrive/2025_COSI_149B_Project1/Teamwork/output/model/added_data/fianl_color_model_alpha0.2_b32_beta0.5_dec2_drop10_e32_enc3_gamma0.3_lat256_losscomposite_lr0.005_20250322_2225/scalers_e32_b32_lr0.005_lat256.pkl'

    # Load model
    print("Loading model...")
    try:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model, X_scaler, y_scaler = load_model(model_dir, scaler_path, device)
        print(f"Model loaded successfully. Using device: {device}")
    except Exception as e:
        print(f"Error loading model: {e}")
        print("\nPlease check the model and scaler paths.")
        return

    # Try to define rgb_dict from user input
    try:
        # Use rgb_dict already defined at the top of the file
        print("Using rgb_dict defined in the code.")
    except Exception as e:
        print(f"Error defining rgb_dict: {e}")
        print("Using externally defined rgb_dict.")

    # Process rgb_dict
    print("Processing color information from rgb_dict...")
    results = process_rgb_dict(rgb_dict, model, X_scaler, y_scaler, device)

    # Results summary
    print("\nColor processing complete")
    for color_name, result in results.items():
        print(f"{color_name}: Observed RGB {tuple(result['input'])} → Predicted RGB {tuple(result['predicted'])}")

# Run example
if __name__ == "__main__":
    main()